## Credit Card Fraud Detection v0.2 -- ToDo - Work In Progress

In [28]:
# import required libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from google.datalab.ml import TensorBoard

import shutil
import pandas as pd
import tensorflow as tf

print(tf.__version__)

1.8.0


In [5]:
# configure logging
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.1f}'.format

In [3]:
OUTDIR = './trained_model/v02'

In [25]:
# makes result reproducible
#np.random.seed(seed=1) 
#tf.random.set_random_seed(seed)

In [6]:
df_train = pd.read_csv('./data/preprocess/creditcard_ohe_header_train.csv')
df_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,...,V12,V14,V16,V17,V18,V19,V21,Amount,Fraud,Normal
0,1.2,-2.7,2.8,-5.8,5.4,-1.1,-1.6,-3.4,-1.4,-2.1,...,-8.5,-13.8,-4.8,-6.4,-1.4,1.3,0.8,-0.4,1,0
1,-0.6,0.6,-0.3,0.5,-0.0,-0.4,0.6,-0.7,1.1,-0.3,...,1.4,-0.9,0.1,-0.5,0.3,1.5,-0.1,-0.2,1,0
2,-1.1,0.6,1.7,-2.7,3.4,2.1,-1.5,1.4,-2.2,-1.9,...,-2.4,-7.5,2.7,7.6,3.1,-3.5,-0.2,-0.4,1,0
3,-0.7,0.2,0.8,-1.3,1.7,-1.1,-1.1,-2.0,-1.0,-3.0,...,-3.9,-4.6,-2.7,-5.9,-0.9,0.3,0.8,-0.3,1,0
4,1.6,0.7,0.8,-3.3,1.0,0.3,-1.0,-1.1,-1.0,-3.0,...,-3.1,-5.4,-2.5,-3.8,-0.8,0.9,0.5,-0.4,1,0


In [7]:
df_train.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,...,V12,V14,V16,V17,V18,V19,V21,Amount,Fraud,Normal
count,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,...,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0
mean,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,1.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
min,-2.0,-28.8,-44.0,-31.8,-4.0,-81.9,-19.6,-35.0,-12.2,-22.6,...,-18.7,-20.0,-16.1,-29.5,-11.3,-8.5,-47.8,-0.4,0.0,0.0
25%,-0.9,-0.5,-0.4,-0.6,-0.6,-0.5,-0.6,-0.4,-0.6,-0.5,...,-0.4,-0.4,-0.5,-0.6,-0.6,-0.6,-0.3,-0.3,0.0,1.0
50%,-0.2,0.0,0.0,0.1,-0.0,-0.0,-0.2,0.0,-0.0,-0.1,...,0.1,0.1,0.1,-0.1,-0.0,0.0,-0.0,-0.3,0.0,1.0
75%,0.9,0.7,0.5,0.7,0.5,0.4,0.3,0.5,0.5,0.4,...,0.6,0.5,0.6,0.5,0.6,0.6,0.3,-0.0,0.0,1.0
max,1.6,1.3,13.4,6.2,11.9,25.1,54.9,96.8,14.2,21.8,...,7.8,11.0,19.7,10.8,5.6,6.9,37.3,103.0,1.0,1.0


In [8]:
df_test = pd.read_csv('./data/preprocess/creditcard_ohe_header_test.csv')
df_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,...,V12,V14,V16,V17,V18,V19,V21,Amount,Fraud,Normal
0,-2.0,-0.7,-0.0,1.7,1.0,-0.3,0.4,0.2,0.3,0.1,...,-0.6,-0.3,-0.5,0.2,0.0,0.5,-0.0,0.2,0,1
1,-2.0,-0.5,-0.1,1.2,-0.6,-0.0,0.9,0.2,-1.3,-0.1,...,0.2,-0.3,-1.2,-0.8,2.4,-1.5,-0.1,0.1,0,1
2,-2.0,-0.6,0.5,1.0,0.3,-0.3,0.1,0.5,0.7,0.7,...,0.5,-1.2,-0.5,-0.3,-0.0,1.0,-0.0,-0.1,0,1
3,-2.0,0.7,-0.7,0.6,-1.0,-1.5,-0.5,-1.2,-1.6,1.5,...,-0.7,-0.1,0.0,0.3,1.0,-0.3,-0.0,-0.3,0,1
4,-2.0,0.6,-0.7,0.3,-0.9,-1.1,-0.6,-0.6,-1.9,1.2,...,-0.2,-0.3,-0.9,1.0,-1.0,-0.8,-0.3,0.1,0,1


In [9]:
df_test.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,...,V12,V14,V16,V17,V18,V19,V21,Amount,Fraud,Normal
count,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,...,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0
mean,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,...,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,1.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
min,-2.0,-20.5,-38.4,-19.0,-3.8,-26.1,-13.6,-19.5,-7.9,-16.9,...,-17.3,-18.0,-14.6,-26.0,-9.3,-8.9,-30.0,-0.3,0.0,0.0
25%,-0.9,-0.5,-0.4,-0.6,-0.6,-0.5,-0.6,-0.5,-0.6,-0.5,...,-0.4,-0.4,-0.5,-0.6,-0.6,-0.6,-0.3,-0.3,0.0,1.0
50%,-0.2,0.0,0.0,0.1,-0.0,-0.0,-0.2,0.0,-0.0,-0.1,...,0.1,0.1,0.1,-0.1,-0.0,0.0,-0.0,-0.3,0.0,1.0
75%,0.9,0.7,0.5,0.7,0.5,0.5,0.3,0.5,0.5,0.4,...,0.6,0.5,0.6,0.5,0.6,0.6,0.2,-0.0,0.0,1.0
max,1.6,1.2,9.3,2.8,11.8,21.2,16.4,30.7,9.4,14.2,...,4.4,7.9,9.5,9.5,6.0,5.9,35.9,76.5,1.0,1.0


In [13]:
df_test[['Fraud','Normal']].head()

,Fraud,Normal
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [15]:
y_test = df_test.Fraud
y_test = pd.concat([y_test, df_test.Normal], axis=1)
y_test.head()

,Fraud,Normal
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [26]:
df_train.as_matrix()

array([[ 1.21132441, -2.66329553,  2.81112529, ..., -0.170521  ,
        -0.24680581,  3.49405355],
       [-0.61968481,  0.60866177, -0.34492202, ..., -0.170521  ,
         4.05175071, -0.28619928],
       [-1.14787229,  0.59104839,  1.72314996, ..., -0.170521  ,
        -0.24680581, -0.28619928],
       ...,
       [ 1.07065411, -0.15421171,  0.35224529, ..., -0.170521  ,
        -0.24680581, -0.28619928],
       [-1.10270197, -2.58673307, -4.1712758 , ..., -0.170521  ,
        -0.24680581, -0.28619928],
       [ 0.85802717,  1.03071135, -0.13725691, ..., -0.170521  ,
        -0.24680581, -0.28619928]])

In [27]:
y_test.as_matrix()

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]])

In [16]:
INPUT_COLUMNS = [
  tf.feature_column.numeric_column("Time", dtype=tf.float32),
  tf.feature_column.numeric_column("V1", dtype=tf.float32),
  tf.feature_column.numeric_column("V2", dtype=tf.float32),
  tf.feature_column.numeric_column("V3", dtype=tf.float32),
  tf.feature_column.numeric_column("V4", dtype=tf.float32),
  tf.feature_column.numeric_column("V5", dtype=tf.float32),
  tf.feature_column.numeric_column("V6", dtype=tf.float32),
  tf.feature_column.numeric_column("V7", dtype=tf.float32),
  tf.feature_column.numeric_column("V9", dtype=tf.float32),
  tf.feature_column.numeric_column("V10", dtype=tf.float32),
  tf.feature_column.numeric_column("V11", dtype=tf.float32),
  tf.feature_column.numeric_column("V12", dtype=tf.float32),
  tf.feature_column.numeric_column("V14", dtype=tf.float32),
  tf.feature_column.numeric_column("V16", dtype=tf.float32),
  tf.feature_column.numeric_column("V17", dtype=tf.float32),
  tf.feature_column.numeric_column("V18", dtype=tf.float32),
  tf.feature_column.numeric_column("V19", dtype=tf.float32),
  tf.feature_column.numeric_column("V21", dtype=tf.float32),
  tf.feature_column.numeric_column("Amount", dtype=tf.float32)
]

In [17]:
ADDITIONAL_COLUMNS = [
  tf.feature_column.numeric_column("V1_", dtype=tf.float32),
  tf.feature_column.numeric_column("V2_", dtype=tf.float32),
  tf.feature_column.numeric_column("V3_", dtype=tf.float32),
  tf.feature_column.numeric_column("V4_", dtype=tf.float32),
  tf.feature_column.numeric_column("V5_", dtype=tf.float32),
  tf.feature_column.numeric_column("V6_", dtype=tf.float32),
  tf.feature_column.numeric_column("V7_", dtype=tf.float32),
  tf.feature_column.numeric_column("V9_", dtype=tf.float32),
  tf.feature_column.numeric_column("V10_", dtype=tf.float32),
  tf.feature_column.numeric_column("V11_", dtype=tf.float32),
  tf.feature_column.numeric_column("V12_", dtype=tf.float32),
  tf.feature_column.numeric_column("V14_", dtype=tf.float32),
  tf.feature_column.numeric_column("V16_", dtype=tf.float32),
  tf.feature_column.numeric_column("V17_", dtype=tf.float32),
  tf.feature_column.numeric_column("V18_", dtype=tf.float32),
  tf.feature_column.numeric_column("V19_", dtype=tf.float32),
  tf.feature_column.numeric_column("V21_", dtype=tf.float32),
  tf.feature_column.numeric_column("Amount_max_fraud", dtype=tf.float32)
]

In [18]:
# Define your feature columns
def create_feature_cols():
  return INPUT_COLUMNS + ADDITIONAL_COLUMNS

In [19]:
def add_more_features(df):
  "Add additional features"
  # ToDo Add additional standardized columns
  df['Amount_max_fraud'] = 1
  df.loc[df.Amount <= 2125.87, 'Amount_max_fraud'] = 0
  df['V1_'] = df.V1.map(lambda x: 1 if x < -3 else 0)
  df['V2_'] = df.V2.map(lambda x: 1 if x > 2.5 else 0)
  df['V3_'] = df.V3.map(lambda x: 1 if x < -4 else 0)
  df['V4_'] = df.V4.map(lambda x: 1 if x > 2.5 else 0)
  df['V5_'] = df.V5.map(lambda x: 1 if x < -4.5 else 0)
  df['V6_'] = df.V6.map(lambda x: 1 if x < -2.5 else 0)
  df['V7_'] = df.V7.map(lambda x: 1 if x < -3 else 0)
  df['V9_'] = df.V9.map(lambda x: 1 if x < -2 else 0)
  df['V10_'] = df.V10.map(lambda x: 1 if x < -2.5 else 0)
  df['V11_'] = df.V11.map(lambda x: 1 if x > 2 else 0)
  df['V12_'] = df.V12.map(lambda x: 1 if x < -2 else 0)
  df['V14_'] = df.V14.map(lambda x: 1 if x < -2.5 else 0)
  df['V16_'] = df.V16.map(lambda x: 1 if x < -2 else 0)
  df['V17_'] = df.V17.map(lambda x: 1 if x < -2 else 0)
  df['V18_'] = df.V18.map(lambda x: 1 if x < -2 else 0)
  df['V19_'] = df.V19.map(lambda x: 1 if x > 1.5 else 0)
  df['V21_'] = df.V21.map(lambda x: 1 if x > 0.6 else 0)
  return df

In [20]:
def transform_feature_cols(df):
  """
  Transform each feature in features so that it has a mean of 0 and standard deviation of 1; 
  This helps with training the neural network.
  """
  features = df.columns.values
  for feature in features:
    if (feature != 'Fraud') and (feature != 'Normal'):
      mean, std = df[feature].mean(), df[feature].std()
      df.loc[:, feature] = (df[feature] - mean) / std
  return df

In [21]:
def train_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = transform_feature_cols(add_more_features(df)),
    y = df[['Fraud','Normal']],
    batch_size = 256,
    num_epochs = None,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [22]:
def eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = transform_feature_cols(add_more_features(df)),
    y = df[['Fraud','Normal']],
    batch_size = 128,
    num_epochs = 1,
    shuffle = False,
    num_threads = 1
  )

In [23]:
def serving_input_fn():
  """Servers prediction service"""
  feature_placeholders = {
      column.name: tf.placeholder(tf.float32, [None]) for column in INPUT_COLUMNS
  }
  features = add_more_features(pd.DataFrame.from_dict(feature_placeholders.copy()))
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [38]:
# ToDo impelement for custom estimator
def dnn_model(img, mode, hparams):
  pass

In [39]:
# ToDo impelement for custom estimator
def transaction_classifier(features, labels, mode, params):
  """Transaction classifier either fraud or normal"""
  pass

In [24]:
def train_and_evaluate(output_dir, num_train_steps):
  """Training and evalucation function"""
  EVAL_INTERVAL = 10
  feature_columns = create_feature_cols()
  estimator = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[37, 18, 9],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=2,
    dropout=0.1,
    model_dir=output_dir)
    
  train_spec=tf.estimator.TrainSpec(input_fn=train_input_fn(df_train), max_steps=num_train_steps)
  # ToDo Fix issue of add engg feature with panda read input functions
  # exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  # eval_spec=tf.estimator.EvalSpec(input_fn=eval_input_fn(df_test), throttle_secs=EVAL_INTERVAL, exporters=exporter)
  eval_spec=tf.estimator.EvalSpec(input_fn=eval_input_fn(df_test), throttle_secs=EVAL_INTERVAL)
  
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [45]:
# Launch tensorboard
TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 44736. Click here to access it.

44736

In [ ]:
# Run the model
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 10000)

In [44]:
#for pid in TensorBoard.list()['pid']:
#  TensorBoard().stop(pid)
#  print('Stopped TensorBoard with pid {}'.format(pid))

<pre>
# Copyright 2018 Atos. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>